In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

import pandas as pd
import geopandas as gpd
from siuba import *

import shared_utils
import datetime as dt

import gcsfs

from calitp_data.storage import get_fs
fs = get_fs()

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/safety_projects/"

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_430/1261345588.py:5: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  im

# Read in TIMS Export

In [2]:
tims = pd.read_parquet(f'{GCS_FILE_PATH}TIMS_Data.parquet')

In [3]:
tims.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929291 entries, 0 to 929290
Data columns (total 80 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   CASE_ID                  929291 non-null  int32  
 1   ACCIDENT_YEAR            929291 non-null  int32  
 2   PROC_DATE                929291 non-null  object 
 3   JURIS                    929291 non-null  object 
 4   COLLISION_DATE           929291 non-null  object 
 5   COLLISION_TIME           929291 non-null  int32  
 6   OFFICER_ID               929283 non-null  object 
 7   REPORTING_DISTRICT       923435 non-null  object 
 8   DAY_OF_WEEK              929291 non-null  int32  
 9   CHP_SHIFT                929291 non-null  int32  
 10  POPULATION               929291 non-null  int32  
 11  CNTY_CITY_LOC            929291 non-null  int32  
 12  SPECIAL_COND             929291 non-null  int32  
 13  BEAT_TYPE                929291 non-null  int32  
 14  CHP_

In [ ]:
tims >> count(_.COLLISION_SEVERITY)

In [ ]:
# keep only small subset of columns
tims_small = (tims 
              >> select(_.CASE_ID,_.ACCIDENT_YEAR,_.COLLISION_DATE,_.COLLISION_TIME,_.COLLISION_SEVERITY,
                                 _.PCF_VIOL_CATEGORY,_.TYPE_OF_COLLISION,_.MVIW,
                                 _.NUMBER_KILLED,_.NUMBER_INJURED,_.PEDESTRIAN_ACCIDENT,_.BICYCLE_ACCIDENT,
                                 _.LATITUDE,_.LONGITUDE,_.POINT_X,_.POINT_Y
                            )
              >> filter(_.COLLISION_SEVERITY<=2) # fatality or severe injury
               )

In [ ]:
tims_small.info()

In [ ]:
tims_small.head(10)

In [ ]:
# make geodataframe w/ relevant columns 
tims_gdf = ((gpd.GeoDataFrame(
    tims_small, geometry=gpd.points_from_xy(tims_small.POINT_X, tims_small.POINT_Y))
           ) >> filter(-_.geometry.is_empty)
           )

In [ ]:
# set a CRS: assume WGS 84? 
tims_gdf = tims_gdf.set_crs('4326')

In [ ]:
# project to match project data
tims_gdf = tims_gdf.to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [ ]:
tims_gdf.explore()